In [1]:
import pyomo.environ as pyo
import numpy as np
import pandas as pd

In [3]:
class SolveAction():
    def __init__(self, model, solver_name):
        self.model = model
        self.solver_name = solver_name

    def solve(self):
        solver_name = self.solver_name
        model = self.model
        
        solver = pyo.SolverFactory(solver_name)
        instance = model.create_instance()
        result = solver.solve(instance)
        instance.A.display()

In [ ]:
class CalculateUsefulFilesAction():
    def __init__(self, useful_requirements, T, D):
        pass

    def calculate(self):
        useful_requirements = self.useful_requirements
        T = self.T

        useful_files = np.dot(useful_requirements, T)
        m = len(useful_files)
        for i in range(m):
            dependencies = self.__calculate_dependencies(i)
            useful_files += dependencies
        return useful_files

    def __calculate_dependencies(self, i):
        useful_requirements = self.useful_requirements
        T = self.T
        D = self.D
        
        if i == 0:
            useful_files = np.dot(useful_requirements, T)
            return np.dot(useful_files, D)
        else:
            dependencies = calculate_dependencies(i - 1)
            return np.dot(dependencies, D)

In [ ]:
class Checker():
    def __init__(self, model):
        self.model = model

    def check(self, vector, check_function_1, check_function_2):
        model = self.model
        
        result = []
        for x in vector:
            f = model.f.add()
            result.append(f)
            constraint_1 = check_function_1(x, f)
            constraint_2 = check_function_2(x, f)
            model.constraints.add(constraint_1)
            model.constraints.add(constraint_2)
        return np.array(result)

In [ ]:
class IncludeChecker():
    def __init__(self, checker, M):
        self.checker = checker
        self.M = M

    def check(self, vector):
        checker = self.checker
        M = self.M
        
        check_include_1 = lambda x, f: (f + 1 / M <= x + 1)
        check_include_2 = lambda x, f: (x <= M * f)

        result = checker.check(vector, check_include_1, check_include_2)
        return result

In [ ]:
class ImplementChecker():
    def __init__(self, checker, M):
        self.checker = checker
        self.M = M

    def check(self, vector):
        checker = self.checker
        M = self.M

        check_implemented_1 = lambda x, f: (x >= f)
        check_implemented_2 = lambda x, f: (x + 1 / M <= 1 + M * f)

        result = checker.check(vector, check_implemented_1, check_implemented_2)
        return result

In [ ]:
class CalculatePluginsAction():
    def __init__(self, calculate_useful_files_action, include_checker, A):
        self.calculate_useful_files_action = calculate_useful_files_action
        self.include_checker = include_checker
        self.A = A

    def calculate(self):
        calculate_useful_files_action = self.calculate_useful_files_action
        include_checker = self.include_checker
        A = self.A

        plugins = np.dot(useful_files, A)
        plugins = include_checker.check(plugins)
        return plugins

In [ ]:
class CalculateDeliveryFilesAction():
    pass

In [ ]:
class Main():

    def __init__(self, m, k, T, D, P, E):
        self.m = m
        self.k = k
        self.T = T
        self.D = D
        self.P = P
        self.E = E
    
    '''
    Входные данные:
    n, m, k, e
    '''
    def run(self):
        m = self.m
        k = self.k
        E = self.E
        
        model = pyo.ConcreteModel(name = 'Optimal decomposition')
        model.constraints = pyo.ConstraintList()
        model.f = pyo.VarList(domain=pyo.Binary)
        set_m = pyo.Set(initialize=range(m))
        set_k = pyo.Set(initialize=range(k))
        model.A = pyo.Var(set_m, set_k, domain=pyo.Binary)
        A = np.array(model.A)

        for row in A:
            model.constraints.add((sum(row) == 1))

        equipment_costs = []
        for useful_requirements in E:
            pass
            
        model.OBJ = pyo.Objective(expr = sum(equipment_costs), sense=pyo.minimize)